In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [18]:
!pip install tensorboardX

     |████████████████████████████████| 124 kB 5.3 MB/s 


In [19]:
import sys
import numpy as np
from tensorboardX import SummaryWriter
sys.path.append('/content/drive/MyDrive/shahbaz/MUNICH')
%load_ext autoreload
% autoreload

In [3]:
%cd /content/drive/MyDrive/shahbaz/MUNICH

/content/drive/MyDrive/Colab_Notebooks/MUNICH


In [4]:
import model

In [5]:
!ls train/data |wc -l
!ls train/seg |wc -l

1000
1000


In [6]:
import dataset

[[[125  94  64]
  [126  96  65]
  [114  90  58]
  ...
  [213 182 141]
  [214 182 145]
  [201 171 135]]

 [[126  93  64]
  [130  99  69]
  [123  97  65]
  ...
  [210 180 137]
  [217 184 146]
  [210 178 142]]

 [[126  93  64]
  [134 101  71]
  [130 102  71]
  ...
  [207 178 133]
  [216 184 144]
  [214 183 146]]

 ...

 [[152 135  96]
  [151 134  96]
  [151 134  97]
  ...
  [105  89  59]
  [103  87  59]
  [101  86  59]]

 [[161 142 104]
  [156 138 100]
  [153 135  96]
  ...
  [110  94  64]
  [108  94  63]
  [108  94  64]]

 [[176 154 118]
  [169 148 111]
  [162 142 103]
  ...
  [116 100  68]
  [115 101  68]
  [117 101  69]]] [[0. 0. 0. ... 0. 0. 0.]
 [0. 1. 1. ... 1. 0. 0.]
 [1. 1. 1. ... 1. 0. 0.]
 ...
 [1. 1. 1. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]]


In [ ]:
# import utils

In [9]:
import torch
import torchvision.transforms as transforms
#import albumentations as A
#from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
from model import UNET


from utils import (
    load_checkpoint,
    save_checkpoint,
    get_data_loader,
    check_accuracy,
    save_predictions_as_imgs,
    compute_f1_iou
)

In [30]:
# Hyperparameters etc.
LEARNING_RATE = 1e-7
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 1
NUM_EPOCHS = 550
NUM_WORKERS = 2
IMAGE_HEIGHT = 32  # 1280 originally
IMAGE_WIDTH = 32  # 1918 originally
PIN_MEMORY = True
LOAD_MODEL = False
TRAIN_IMG_DIR = "/content/drive/MyDrive/shahbaz/MUNICH/train/data"
TRAIN_MASK_DIR = "/content/drive/MyDrive/shahbaz/MUNICH/train/seg" 
VAL_IMG_DIR = "/content/drive/MyDrive/shahbaz/MUNICH/val/data/" 
VAL_MASK_DIR = "/content/drive/MyDrive/shahbaz/MUNICH/val/seg/"

In [31]:
def train_fn(loader, model, optimizer, loss_fn):
    loop = tqdm(loader)

    for batch_idx, (data, targets) in enumerate(loop):
        data = data.to(device=DEVICE)
        targets = targets.float().to(device=DEVICE)

        # forward
      
        with torch.cuda.amp.autocast():
            predictions = model(data)
            loss = loss_fn(predictions, targets)

  
        #predictions = model(data)
        # loss = loss_fn(predictions, targets)     
 # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
       # scaler.scale(loss).backward()
       # scaler.step(optimizer)
       # scaler.update()

        # update tqdm loop
        loop.set_postfix(loss=loss.item())

In [32]:
mean=[0.0, 0.0, 0.0]
std=[1.0, 1.0, 1.0]
train_transform = transforms.Compose([transforms.ToTensor()])
val_transforms = transforms.Compose([transforms.ToTensor()])
model = UNET(in_channels=3, out_channels=1).to(DEVICE)
loss_fn = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [33]:
# old loader function
# train_loader, val_loader = get_loaders(
#     TRAIN_IMG_DIR,
#     TRAIN_MASK_DIR,
#     VAL_IMG_DIR,
#     VAL_MASK_DIR,
#     BATCH_SIZE,
#     train_transform,
#     val_transforms,
#     NUM_WORKERS,
#     PIN_MEMORY,
    
# )

train_loader = get_data_loader(
    TRAIN_IMG_DIR,
    TRAIN_MASK_DIR,
    BATCH_SIZE,
    train_transform,
    split="train",
    num_workers=NUM_WORKERS,
    pin_memory=PIN_MEMORY,
)
val_loader = get_data_loader(
    VAL_IMG_DIR,
    VAL_MASK_DIR,
    BATCH_SIZE,
    val_transforms,
    "val",
    NUM_WORKERS,
    PIN_MEMORY,
)

## Train Loop

In [ ]:
writer = SummaryWriter('logs', flush_secs=1)

In [ ]:
# torch.autograd.set_detect_anomaly(True) #to throw exception if nan values are encountered
if LOAD_MODEL:
    load_checkpoint(torch.load("my_checkpoint.pth.tar"), model)


#check_accuracy(val_loader, model, device=DEVICE)
# scaler = torch.cuda.amp.GradScaler()

best_loss = np.inf 

print(f"Training for {NUM_EPOCHS} epochs with batch size {BATCH_SIZE} and learning rate {LEARNING_RATE}")
for epoch in range(NUM_EPOCHS):

    model.train()
    train_fn(train_loader, model, optimizer, loss_fn)


    # validation loss computation
    val_loss = 0
    with torch.no_grad():
        model.eval()
        for data, targets in val_loader:
            data = data.to(device=DEVICE)
            targets = targets.float().to(device=DEVICE)

            
            predictions = model(data)
            val_loss += loss_fn(predictions, targets)


        avg_loss = val_loss/len(val_loader)
        writer.add_scalar(f'LR_{LEARNING_RATE}_BS_{BATCH_SIZE}/val', avg_loss, epoch)
    print(f"epoch {epoch}, loss={avg_loss}")
    if avg_loss < best_loss:
        best_loss = avg_loss
        best_model = model.state_dict()
        
        print(f"Saving best loss: {best_loss:.6f}")
        # save model
        checkpoint = {
            "state_dict": model.state_dict(),
            "optimizer":optimizer.state_dict(),
        }
        checkpoint_file_name = f"LR_{LEARNING_RATE}_BS_{BATCH_SIZE}_epoch_{epoch}_loss_{avg_loss:.6f}.pth.tar"
        save_checkpoint(checkpoint, checkpoint_file_name)

    # check accuracy
    #check_accuracy(val_loader, model, device=DEVICE)

    if (epoch+1)%10==0:
        check_accuracy(val_loader, model, device=DEVICE)

# print some examples to a folder
# save_predictions_as_imgs(
#     val_loader, model, folder=f"/content/drive/MyDrive/shahbaz/MUNICH/SI_epo_100_BS_{BATCH_SIZE}_LR_{LEARNING_RATE}/", device=DEVICE
# )
writer.close()

Training for 550 epochs with batch size 1 and learning rate 1e-07


  1%|          | 8/1000 [00:04<09:58,  1.66it/s, loss=0.718]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-71783381b3a8>", line 15, in <module>
    train_fn(train_loader, model, optimizer, loss_fn)
  File "<ipython-input-10-ab06c56d4757>", line 4, in train_fn
    for batch_idx, (data, targets) in enumerate(loop):
  File "/usr/local/lib/python3.7/dist-packages/tqdm/std.py", line 1180, in __iter__
    for obj in iterable:
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 521, in __next__
    data = self._next_data()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1186, in _next_data
    idx, data = self._get_data()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1142, in _get_data
    success, data = self._try_get_data()
  File "/usr/local/lib/python3.7/dist-packages/to

KeyboardInterrupt: ignored

## Evaluation

#### BS=8

##### LR= 1e-4, epochs = 100

In [ ]:
best_model = "checkpoints/LR_0.0001_BS_8_epoch_20_loss_0.489122.pth.tar"
bm = torch.load(best_model)
load_checkpoint(bm, model)

In [ ]:
check_accuracy(val_loader, model, device=DEVICE)

**Summary:**
- After 100 epochs:
    * loss = 0.5672
    * accuracy = 76.83
    * DICE score = 0.5246
- Lowest loss model:
    * epoch = 20
    * loss = 0.4891
    * accuracy = 77.17
    * DICE score = 0.5529


##### LR= 1e-5, epochs = 350

In [ ]:
best_model = "checkpoints/LR_1e-05_BS_8_epoch_59_loss_0.543724.pth.tar"
bm = torch.load(best_model)
load_checkpoint(bm, model)

In [ ]:
check_accuracy(val_loader, model, device=DEVICE)

**Summary:**
- After 350 epochs:
    * loss = 0.9123
    * accuracy = 73.72
    * DICE score = 0.4891
- Lowest loss model:
    * epoch = 59
    * loss = 0.5437
    * accuracy = 73.54
    * DICE score = 0.4762


##### LR= 1e-6, epochs = 350

In [ ]:
best_model = "checkpoints/LR_1e-06_BS_8_epoch_347_loss_0.556644.pth.tar"
bm = torch.load(best_model)
load_checkpoint(bm, model)

In [ ]:
check_accuracy(val_loader, model, device=DEVICE)

**Summary:**
- After 350 epochs:
    * loss = 0.5571
    * accuracy = 70.98
    * DICE score = 0.0933
- Lowest loss model:
    * epoch = 347
    * loss = 0.5437
    * accuracy = 71.18
    * DICE score = 0.1126
    * curve shows should have been trained more



##### LR= 1e-7, epochs = 550

In [ ]:
best_model = "checkpoints/LR_1e-07_BS_8_epoch_450_loss_0.596443.pth.tar"
bm = torch.load(best_model)
load_checkpoint(bm, model)

In [ ]:
check_accuracy(val_loader, model, device=DEVICE)

**Summary:**
- After 550 epochs:
    * loss = 0.6137
    * accuracy = 68.10
    * DICE score = 0.5248
- Lowest loss model:
    * epoch = 450
    * loss = 0.0.5964
    * accuracy = 69.68
    * DICE score = 0.5238
    * curve shows could have been


In [ ]:
%reload_ext tensorboard
%tensorboard --logdir ./logs

#### BS=4


##### LR= 1e-4, epochs = 100

In [ ]:
checkpoint_file_name

'LR_0.0001_BS_4_epoch_4_loss_0.478127.pth.tar'

In [ ]:
best_model = "checkpoints/LR_0.0001_BS_4_epoch_4_loss_0.478127.pth.tar"
bm = torch.load(best_model)
load_checkpoint(bm, model)

=> Loading checkpoint


In [ ]:
check_accuracy(val_loader, model, device=DEVICE)

Got 10249820/13107200 with acc 78.20
Dice score: 0.5277642607688904


**Summary:**
- After 100 epochs:
    * loss = 0.9203
    * accuracy = 72.09
    * DICE score = 0.1576
- Lowest loss model:
    * epoch = 4
    * loss = 0.4781
    * accuracy = 78.2
    * DICE score = 0.5278


##### LR= 1e-5, epochs = 350

In [ ]:
best_model = "checkpoints/LR_1e-05_BS_4_epoch_12_loss_0.523466.pth.tar"
bm = torch.load(best_model)
load_checkpoint(bm, model)

=> Loading checkpoint


In [ ]:
check_accuracy(val_loader, model, device=DEVICE)

Got 9820665/13107200 with acc 74.93
Dice score: 0.5123778581619263


**Summary:**
- After 350 epochs:
    * loss = 1.7465
    * accuracy = 74.63
    * DICE score = 0.5103
- Lowest loss model:
    * epoch = 12
    * loss = 0.5235
    * accuracy = 74.93
    * DICE score = 0.5124


##### LR= 1e-6, epochs = 350

In [ ]:
best_model = "checkpoints/LR_1e-06_BS_4_epoch_148_loss_0.545599.pth.tar"
bm = torch.load(best_model)
load_checkpoint(bm, model)

=> Loading checkpoint


In [ ]:
check_accuracy(val_loader, model, device=DEVICE)

Got 9652638/13107200 with acc 73.64
Dice score: 0.4847889244556427


**Summary:**
- After 350 epochs:
    * loss = 0.61
    * accuracy = 71.84
    * DICE score = 0.4984
- Lowest loss model:
    * epoch = 148
    * loss = 0.5456
    * accuracy = 73.64
    * DICE score = 0.4847


##### LR= 1e-7, epochs = 550

In [ ]:
best_model = "checkpoints/LR_1e-07_BS_4_epoch_538_loss_0.554473.pth.tar"
bm = torch.load(best_model)
load_checkpoint(bm, model)

=> Loading checkpoint


In [ ]:
check_accuracy(val_loader, model, device=DEVICE)

Got 9527415/13107200 with acc 72.69
Dice score: 0.5013424754142761


**Summary:**
- After 550 epochs:
    * loss = 0.5606
    * accuracy = 72.05
    * DICE score = 0.5142
- Lowest loss model:
    * epoch = 538
    * loss = 0.5545
    * accuracy = 72.69
    * DICE score = 0.5013
    * (curve shows could have been trained further)


#### BS=2


##### LR= 1e-4, epochs = 100

In [ ]:
best_model = "checkpoints/LR_0.0001_BS_2_epoch_5_loss_0.464750.pth.tar"
bm = torch.load(best_model)
load_checkpoint(bm, model)

=> Loading checkpoint


In [ ]:
check_accuracy(val_loader, model, device=DEVICE)

Got 10288153/13107200 with acc 78.49
Dice score: 0.5671282410621643


**Summary:**
- After 100 epochs:
    * loss = NaN
    * accuracy = 70.15
    * DICE score = 0.0
- Lowest loss model:
    * epoch = 5
    * loss = 0.4647
    * accuracy = 78.49
    * DICE score = 0.5671


##### LR= 1e-5, epochs = 350

In [ ]:
best_model = "checkpoints/LR_1e-05_BS_2_epoch_3_loss_0.514701.pth.tar"
bm = torch.load(best_model)
load_checkpoint(bm, model)

=> Loading checkpoint


In [ ]:
check_accuracy(val_loader, model, device=DEVICE)

Got 9922473/13107200 with acc 75.70
Dice score: 0.4626854360103607


**Summary:**
- After 350 epochs:
    * loss = 2.5155
    * accuracy = 75.39
    * DICE score = 0.5052
- Lowest loss model:
    * epoch = 3
    * loss = 0.5147
    * accuracy = 75.70
    * DICE score = 0.4627


##### LR= 1e-6, epochs = 350

In [ ]:
best_model = "checkpoints/LR_1e-06_BS_2_epoch_35_loss_0.531021.pth.tar"
bm = torch.load(best_model)
load_checkpoint(bm, model)

=> Loading checkpoint


In [ ]:
check_accuracy(val_loader, model, device=DEVICE)

Got 9781481/13107200 with acc 74.63
Dice score: 0.4275987148284912


**Summary:**
- After 350 epochs:
    * loss = 0.776
    * accuracy = 72.18
    * DICE score = 0.5072
- Lowest loss model:
    * epoch = 35
    * loss = 0.5310
    * accuracy = 74.63
    * DICE score = 0.4276


##### LR= 1e-7, epochs = 550

In [ ]:
best_model = "checkpoints/LR_1e-07_BS_2_epoch_450_loss_0.535920.pth.tar"
bm = torch.load(best_model)
load_checkpoint(bm, model)

=> Loading checkpoint


In [ ]:
check_accuracy(val_loader, model, device=DEVICE)

Got 9705656/13107200 with acc 74.05
Dice score: 0.467281699180603


**Summary:**
- After 550 epochs:
    * loss = 0.5419
    * accuracy = 73.39
    * DICE score = 0.4953
- Lowest loss model:
    * epoch = 450
    * loss = 0.5359
    * accuracy = 74.05
    * DICE score = 0.4273


#### BS=1


##### LR= 1e-4, epochs = 100

In [ ]:
best_model = "checkpoints/LR_0.0001_BS_1_epoch_3_loss_0.478231.pth.tar"
bm = torch.load(best_model)
load_checkpoint(bm, model)

=> Loading checkpoint


In [ ]:
check_accuracy(val_loader, model, device=DEVICE)

Got 10227299/13107200 with acc 78.03
Dice score: 0.5130472183227539


**Summary:**
- After 100 epochs:
    * loss = NaN
    * accuracy = 70.15
    * DICE score = 0.0
- Lowest loss model:
    * epoch = 3
    * loss = 0.4782
    * accuracy = 78.03
    * DICE score = 0.513


##### LR= 1e-5, epochs = 350

In [ ]:
best_model = "checkpoints/LR_1e-05_BS_1_epoch_2_loss_0.519166.pth.tar"
bm = torch.load(best_model)
load_checkpoint(bm, model)

=> Loading checkpoint


In [ ]:
check_accuracy(val_loader, model, device=DEVICE)

Got 9867297/13107200 with acc 75.28
Dice score: 0.45672187209129333


**Summary:**
- After 350 epochs:
    * loss = 3.2627
    * accuracy = 74.39
    * DICE score = 0.4834
- Lowest loss model:
    * epoch = 2
    * loss = 0.5192
    * accuracy = 75.28
    * DICE score = 0.4567


##### LR= 1e-6, epochs = 350

In [ ]:
best_model = "checkpoints/LR_1e-06_BS_1_epoch_31_loss_0.578217.pth.tar"
bm = torch.load(best_model)
load_checkpoint(bm, model)

=> Loading checkpoint


In [ ]:
check_accuracy(val_loader, model, device=DEVICE)

Got 9354081/13107200 with acc 71.37
Dice score: 0.44403010606765747


**Summary:**
- After 350 epochs:
    * loss = 1.4651
    * accuracy = 68.35
    * DICE score = 0.4229
- Lowest loss model:
    * epoch = 31
    * loss = 0.5782
    * accuracy = 71.37
    * DICE score = 0.444


##### LR= 1e-7, epochs = 550

In [ ]:
best_model = "checkpoints/LR_1e-07_BS_1_epoch_100_loss_0.552152.pth.tar"
bm = torch.load(best_model)
load_checkpoint(bm, model)

=> Loading checkpoint


In [ ]:
check_accuracy(val_loader, model, device=DEVICE)

Got 9521356/13107200 with acc 72.64
Dice score: 0.4337500333786011


**Summary:**
- After 550 epochs:
    * loss = 0.7419
    * accuracy = 66.64
    * DICE score = 0.4212
- Lowest loss model:
    * epoch = 100
    * loss = 0.5522
    * accuracy = 72.64
    * DICE score = 0.4338


### Tensorboard graphs for all experiments

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir ./logs

### F1, IOU

In [ ]:
best_models = ['LR_0.0001_BS_8_epoch_20_loss_0.489122.pth.tar', 'LR_1e-05_BS_8_epoch_59_loss_0.543724.pth.tar', 
               'LR_1e-06_BS_8_epoch_347_loss_0.556644.pth.tar', 'LR_1e-07_BS_8_epoch_450_loss_0.596443.pth.tar',
               'LR_0.0001_BS_4_epoch_4_loss_0.478127.pth.tar', 'LR_1e-05_BS_4_epoch_12_loss_0.523466.pth.tar',
               'LR_1e-06_BS_4_epoch_148_loss_0.545599.pth.tar', 'LR_1e-07_BS_4_epoch_538_loss_0.554473.pth.tar',
               'LR_0.0001_BS_2_epoch_5_loss_0.464750.pth.tar', 'LR_1e-05_BS_2_epoch_3_loss_0.514701.pth.tar',
               'LR_1e-06_BS_2_epoch_35_loss_0.531021.pth.tar', 'LR_1e-07_BS_2_epoch_450_loss_0.535920.pth.tar',
               'LR_0.0001_BS_1_epoch_3_loss_0.478231.pth.tar', 'LR_1e-05_BS_1_epoch_2_loss_0.519166.pth.tar',
               'LR_1e-06_BS_1_epoch_31_loss_0.578217.pth.tar', 'LR_1e-07_BS_1_epoch_100_loss_0.552152.pth.tar']# add in this list

In [ ]:
best_model_ious = []
best_model_f1 = []

for i,model_name in enumerate(best_models):
    best_model = f"checkpoints/{model_name}"
    print(f"Evaluating model {i+1}: {best_model}")
    bm = torch.load(best_model)
    load_checkpoint(bm, model)

    f1, iou = compute_f1_iou(val_loader, model, device=DEVICE)
    print(f"IOU: {iou}, f1: {f1}\n\n")

    best_model_ious.append(iou)
    best_model_f1.append(f1)

best_model_ious = np.array(best_model_ious)
best_model_f1 = np.array(best_model_f1)

Evaluating model 1: checkpoints/LR_0.0001_BS_8_epoch_20_loss_0.489122.pth.tar
=> Loading checkpoint
IOU: 0.3812102042212823, f1: 0.5472084497570103


Evaluating model 2: checkpoints/LR_1e-05_BS_8_epoch_59_loss_0.543724.pth.tar
=> Loading checkpoint
IOU: 0.31207024802706934, f1: 0.47159809326773205


Evaluating model 3: checkpoints/LR_1e-06_BS_8_epoch_347_loss_0.556644.pth.tar
=> Loading checkpoint
IOU: 0.05809608675923514, f1: 0.10682794171870609


Evaluating model 4: checkpoints/LR_1e-07_BS_8_epoch_450_loss_0.596443.pth.tar
=> Loading checkpoint
IOU: 0.3421241294022005, f1: 0.506097577056002


Evaluating model 5: checkpoints/LR_0.0001_BS_4_epoch_4_loss_0.478127.pth.tar
=> Loading checkpoint
IOU: 0.36098460565115004, f1: 0.5247090665074409


Evaluating model 6: checkpoints/LR_1e-05_BS_4_epoch_12_loss_0.523466.pth.tar
=> Loading checkpoint
IOU: 0.34448578619825837, f1: 0.507648504495385


Evaluating model 7: checkpoints/LR_1e-06_BS_4_epoch_148_loss_0.545599.pth.tar
=> Loading checkpoint

In [ ]:
ind = np.argsort(best_model_ious)
ind = ind[::-1]
print(f"Top models by ious:")
for i in ind:
    print(f"{best_models[i]}, IOU={best_model_ious[i]}")

Top models by ious:
LR_0.0001_BS_2_epoch_5_loss_0.464750.pth.tar, IOU=0.4003343703521684
LR_0.0001_BS_8_epoch_20_loss_0.489122.pth.tar, IOU=0.3812102042212823
LR_0.0001_BS_4_epoch_4_loss_0.478127.pth.tar, IOU=0.36098460565115004
LR_0.0001_BS_1_epoch_3_loss_0.478231.pth.tar, IOU=0.3504424184590604
LR_1e-05_BS_4_epoch_12_loss_0.523466.pth.tar, IOU=0.34448578619825837
LR_1e-07_BS_8_epoch_450_loss_0.596443.pth.tar, IOU=0.3421241294022005
LR_1e-07_BS_4_epoch_538_loss_0.554473.pth.tar, IOU=0.33326371636279717
LR_1e-06_BS_4_epoch_148_loss_0.545599.pth.tar, IOU=0.3207774152189836
LR_1e-05_BS_8_epoch_59_loss_0.543724.pth.tar, IOU=0.31207024802706934
LR_1e-07_BS_2_epoch_450_loss_0.535920.pth.tar, IOU=0.307369806588767
LR_1e-05_BS_2_epoch_3_loss_0.514701.pth.tar, IOU=0.3045108396810133
LR_1e-05_BS_1_epoch_2_loss_0.519166.pth.tar, IOU=0.30048008005742394
LR_1e-06_BS_1_epoch_31_loss_0.578217.pth.tar, IOU=0.28849390105620076
LR_1e-07_BS_1_epoch_100_loss_0.552152.pth.tar, IOU=0.28082808020908856
LR_1

In [ ]:
find = np.argsort(best_model_f1)
ind = ind[::-1]
print(f"Top models by F1 score:")
for i in ind:
    print(f"{best_models[i]}, F1={best_model_f1[i]}")

Top models by F1 score:
LR_0.0001_BS_2_epoch_5_loss_0.464750.pth.tar, F1=0.5660587087317223
LR_0.0001_BS_8_epoch_20_loss_0.489122.pth.tar, F1=0.5472084497570103
LR_0.0001_BS_4_epoch_4_loss_0.478127.pth.tar, F1=0.5247090665074409
LR_0.0001_BS_1_epoch_3_loss_0.478231.pth.tar, F1=0.5130847054260598
LR_1e-05_BS_4_epoch_12_loss_0.523466.pth.tar, F1=0.507648504495385
LR_1e-07_BS_8_epoch_450_loss_0.596443.pth.tar, F1=0.506097577056002
LR_1e-07_BS_4_epoch_538_loss_0.554473.pth.tar, F1=0.4956144170977849
LR_1e-06_BS_4_epoch_148_loss_0.545599.pth.tar, F1=0.48123891183685485
LR_1e-05_BS_8_epoch_59_loss_0.543724.pth.tar, F1=0.47159809326773205
LR_1e-07_BS_2_epoch_450_loss_0.535920.pth.tar, F1=0.46438004589974224
LR_1e-05_BS_2_epoch_3_loss_0.514701.pth.tar, F1=0.4612552148583455
LR_1e-05_BS_1_epoch_2_loss_0.519166.pth.tar, F1=0.45672214176928916
LR_1e-06_BS_1_epoch_31_loss_0.578217.pth.tar, F1=0.4440301217700916
LR_1e-07_BS_1_epoch_100_loss_0.552152.pth.tar, F1=0.4337501375199148
LR_1e-06_BS_2_epoc

### Inference from Best Model

In [ ]:
best_model = "checkpoints/LR_0.0001_BS_2_epoch_5_loss_0.464750.pth.tar"
bm = torch.load(best_model)
load_checkpoint(bm, model)

=> Loading checkpoint


In [ ]:
output_dir = f"output/{best_model.split('/')[-1][:-8]}"
output_dir

'output/LR_0.0001_BS_2_epoch_5_loss_0.464750'

In [ ]:
save_predictions_as_imgs(val_loader, model, output_dir)

## Evaluation and inference on Amsterdam dataset

In [34]:
BATCH_SIZE = 1

In [35]:
img_dir = "Amsterdam/data"
mask_dir = "Amsterdam/seg"

test_loader = get_data_loader(
    img_dir,
    mask_dir,
    BATCH_SIZE,
    val_transforms,
    "test",
    NUM_WORKERS,
    PIN_MEMORY,
)

In [36]:
best_model = "checkpoints/LR_0.0001_BS_2_epoch_5_loss_0.464750.pth.tar"
bm = torch.load(best_model)
load_checkpoint(bm, model)



=> Loading checkpoint


In [37]:
# from sklearn.metrics import f1_score, confusion_matrix
# instance_f1_scores = []
# instance_ious = []
# model.eval()

# with torch.no_grad():
#     for x, y in test_loader: 
#         x = x.to(DEVICE)
#         y = y.to(DEVICE)
#         y_pred = torch.sigmoid(model(x))
#         y_pred_class = (y_pred>0.5).int()
#         f1 = f1_score(y.cpu().ravel(), y_pred_class.cpu().ravel(), labels=[0,1])
#         instance_f1_scores.append(f1)

#         tn, fp, fn, tp = confusion_matrix(y.cpu().ravel(), y_pred_class.cpu().ravel(), labels=[0,1]).ravel()
#         iou = tp/(tp+fn+fp + 1e-8) 
#         instance_ious.append(iou)

In [39]:
f1, iou = compute_f1_iou(test_loader, model, device=DEVICE)
print(f"F1: {f1}, IOU: {iou}")

FileNotFoundError: ignored

In [ ]:
# inference on images
output_dir = f"output/amsterdam_{best_model.split('/')[-1][:-8]}"
save_predictions_as_imgs(test_loader, model, output_dir)

## Evaluation and inference on Munich dataset

In [ ]:
BATCH_SIZE = 1

In [ ]:
img_dir = "test/data"
mask_dir = "test/seg"

test_loader = get_data_loader(
    img_dir,
    mask_dir,
    BATCH_SIZE,
    val_transforms,
    "test",
    NUM_WORKERS,
    PIN_MEMORY,
)

In [ ]:
best_model = "checkpoints/LR_0.0001_BS_2_epoch_5_loss_0.464750.pth.tar"
# bm = torch.load(best_model)
bm = torch.load(best_model, map_location=torch.device('cpu'))
load_checkpoint(bm, model)

=> Loading checkpoint


In [ ]:
f1, iou = compute_f1_iou(test_loader, model, device=DEVICE)
print(f"F1: {f1}, IOU: {iou}")

F1: 0.5866751001620263, IOU: 0.41852044896569013


In [ ]:
!ls output/munich_test_LR_0.0001_BS_2_epoch_5_loss_0.464750/pred* 

output/munich_test_LR_0.0001_BS_2_epoch_5_loss_0.464750/pred_0.png
output/munich_test_LR_0.0001_BS_2_epoch_5_loss_0.464750/pred_100.png
output/munich_test_LR_0.0001_BS_2_epoch_5_loss_0.464750/pred_101.png
output/munich_test_LR_0.0001_BS_2_epoch_5_loss_0.464750/pred_102.png
output/munich_test_LR_0.0001_BS_2_epoch_5_loss_0.464750/pred_103.png
output/munich_test_LR_0.0001_BS_2_epoch_5_loss_0.464750/pred_104.png
output/munich_test_LR_0.0001_BS_2_epoch_5_loss_0.464750/pred_105.png
output/munich_test_LR_0.0001_BS_2_epoch_5_loss_0.464750/pred_106.png
output/munich_test_LR_0.0001_BS_2_epoch_5_loss_0.464750/pred_107.png
output/munich_test_LR_0.0001_BS_2_epoch_5_loss_0.464750/pred_108.png
output/munich_test_LR_0.0001_BS_2_epoch_5_loss_0.464750/pred_109.png
output/munich_test_LR_0.0001_BS_2_epoch_5_loss_0.464750/pred_10.png
output/munich_test_LR_0.0001_BS_2_epoch_5_loss_0.464750/pred_110.png
output/munich_test_LR_0.0001_BS_2_epoch_5_loss_0.464750/pred_111.png
output/munich_test_LR_0.0001_BS_2_epo

In [ ]:
# inference on images
output_dir = f"output/munich_test_{best_model.split('/')[-1][:-8]}"
save_predictions_as_imgs(test_loader, model, output_dir, device=DEVICE)

KeyboardInterrupt: ignored